In [2]:
import numpy as np
# Compute the one-dimensional discrete Fourier Transform
import numpy.fft as fft
import pandas as pd

from queue import PriorityQueue
from numba import njit

from sklearn.metrics.pairwise import paired_euclidean_distances
from clasp import extract_clasp_cps

# the sliding windows for a time series and a window size
def sliding_window(ts, window):
    shape = ts.shape[:-1] + (ts.shape[-1] - window + 1, window)
    strides = ts.strides + (ts.strides[-1],)
    return np.lib.stride_tricks.as_strided(ts, shape=shape, strides=strides)


# the sliding dot product between a query subsequence and a time series
def slidingDotProduct(query, ts):
    m = len(query)
    n = len(ts)

    ts_add = 0
    if n % 2 == 1:
        ts = np.insert(ts, 0, 0)
        ts_add = 1

    q_add = 0
    if m % 2 == 1:
        query = np.insert(query, 0, 0)
        q_add = 1

    query = query[::-1]
    query = np.pad(query, (0, n - m + ts_add - q_add), 'constant')
    trim = m - 1 + ts_add
    dot_product = fft.irfft(fft.rfft(ts) * fft.rfft(query))
    return dot_product[trim:]


# the sliding mean and std for a time series and a window size
def sliding_mean_std(TS, m):
    s = np.insert(np.cumsum(TS), 0, 0)
    sSq = np.insert(np.cumsum(TS ** 2), 0, 0)
    segSum = s[m:] - s[:-m]
    segSumSq = sSq[m:] - sSq[:-m]
    movmean = segSum / m
    movstd = np.sqrt(segSumSq / m - (segSum / m) ** 2)
    return [movmean, movstd]

In [23]:
# Done
# kNN indices with dot-product / no-loops for a time series, a window size and k neighbours
def compute_distances_iterative(TS, m, k):
    l = len(TS) - m + 1
    knns = np.zeros(shape=(l, k), dtype=np.int64)

    dot_prev = None
    means, stds = sliding_mean_std(TS, m)
    print('Means:', means)
    print('Stds:', stds)

    for order in range(0, l):
        # first iteration O(n log n)
        if order == 0:
            dot_first = slidingDotProduct(TS[:m], TS)
            # dot_first = np.dot(X[order,:], X.T)
            dot_rolled = dot_first
            print('dot_first:', dot_first)
        # O(1) further operations
        else:
            dot_rolled = np.roll(dot_prev, 1) + TS[order + m - 1] * TS[m - 1:l + m] - TS[order - 1] * np.roll(TS[:l], 1)
            dot_rolled[0] = dot_first[order]

        x_mean = means[order]
        x_std = stds[order]

        dist = 2 * m * (1 - (dot_rolled - m * means * x_mean) / (m * stds * x_std))

        # self-join: exclusion zone
        trivialMatchRange = (int(max(0, order - np.round(m / 2, 0))), int(min(order + np.round(m / 2 + 1, 0), l)))
        dist[trivialMatchRange[0]:trivialMatchRange[1]] = np.inf

        idx = np.argpartition(dist, k)

        knns[order, :] = idx[:k]
        dot_prev = dot_rolled

    return knns

In [24]:
TS = np.array([1,2,3,1,5,6,3,2])
m = 2
k = 2
knns = compute_distances_iterative(TS, m, k)
print(knns)

Means: [1.5 2.5 2.  3.  5.5 4.5 2.5]
Stds: [0.5 0.5 1.  2.  0.5 1.5 0.5]
dot_first: [ 5.  8.  5. 11. 17. 12.  7.]
[[3 4]
 [3 4]
 [5 6]
 [0 1]
 [0 1]
 [2 1]
 [2 1]]


### Experiment for idea: distribution

#### Step 1: Compute knn_mask of each window with only left part of a certain window index 

In [25]:
# kNN indices with dot-product / no-loops for a time series, a window size and k neighbours
def compute_distances_iterative_left(TS, m, k):
    l = len(TS) - m + 1
    knns = np.zeros(shape=(l, k), dtype=np.int64)

    dot_prev = None
    means, stds = sliding_mean_std(TS, m)
    #print('Means:', means)
    #print('Stds:', stds)

    # modified part
    for order in range(0, l):
        # first iteration O(n log n)
        if order == 0:
            dot_first = slidingDotProduct(TS[:m], TS[:m])
            # dot_first = np.dot(X[order,:], X.T)
            dot_rolled = dot_first
        # O(1) further operations
        else:
            dot_rolled = np.roll(dot_prev, 1) + TS[order + m - 1] * TS[m - 1:order + m] - TS[order - 1] * np.roll(TS[:order], 1)
            dot_rolled[0] = dot_first[order]

        x_mean = means[order]
        x_std = stds[order]

        dist = 2 * m * (1 - (dot_rolled - m * means * x_mean) / (m * stds * x_std))

        # self-join: exclusion zone
        trivialMatchRange = (int(max(0, order - np.round(m / 2, 0))), int(min(order + np.round(m / 2 + 1, 0), l)))
        dist[trivialMatchRange[0]:trivialMatchRange[1]] = np.inf

        idx = np.argpartition(dist, k)

        knns[order, :] = idx[:k]
        dot_prev = dot_rolled

    return knns

In [26]:
TS = np.array([1,2,3,1,5,6,3,2])
m = 2
k = 2
knns_left = compute_distances_iterative_left(TS, m, k)
print(knns_left)

IndexError: index 1 is out of bounds for axis 0 with size 1